# Organellar IP Workflow Tutorial

In [ ]:
import grassp as gr
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import pandas
import numpy as np

# Larger dpi to make plots larger
plt.rcParams["figure.dpi"] = 250

In [ ]:
# adata = sc.read_h5ad("hein2024_raw.h5ad")
adata = gr.datasets.hein_2024(enrichment="raw", include_whole_proteome=True)
adata

## Annotation

In [ ]:
gr.pp.add_markers(
    adata,
    species="hsap",
)
adata.obs.head()

In [ ]:
adata.var.subcellular_enrichment.unique()

## Filtering

In [ ]:
# Remove known contaminants and Decoy proteins from the proteins (based on the MaxQuant output)
print(f"Before n_proteins x n_samples: {adata.shape}")
gr.pp.remove_contaminants(
    adata,
    filter_columns=["Only identified by site", "Reverse", "Potential contaminant"],
    filter_value="+",
)
print(f"After n_proteins x n_samples:: {adata.shape}")

In [ ]:
# Use paired whole proteome data to filter proteins that are low abundance in the cell
print(f"Before n_proteins x n_samples: {adata.shape}")
adata_proteome = adata[:, adata.var.subcellular_enrichment == "PROTEOME"]
protein_mask, n_per_protein = gr.pp.filter_proteins(
    adata_proteome, min_samples=2, inplace=False
)
adata = adata[protein_mask, :]
print(f"After n_proteins x n_samples:: {adata.shape}")

In [ ]:
print(f"Before n_proteins x n_samples: {adata.shape}")
gr.pp.filter_proteins_per_replicate(
    adata,
    grouping_columns=["covariate_Bait"],
    min_replicates=3,
    inplace=True,
)
print(f"After n_proteins x n_samples:: {adata.shape}")

## Transformation and imputation

In [ ]:
# Anndata has a useful layers feature that allows to keep different transformed versions of the data.
adata.layers["LFQ"] = adata.X.copy()
sc.pp.log1p(adata)
# Notice that there is now an additional layer called "LFQ" that stores the untansformed lfq values
# The log transformed data is stored in the "X" layer
adata

In [ ]:
# Impute the missing values with a gaussian distribution at the lower tail of the intensity distribution
adata.layers["log_unimputed"] = adata.X.copy()
gr.pp.impute_gaussian(adata)
_ = plt.hist(adata.X.flatten(), bins=100, alpha=0.5)
_ = plt.hist(adata.layers["log_unimputed"].flatten(), bins=100, alpha=0.5)

## Calculate enrichment

Split up dataset into IP fractions and NOC because the enrichment is different.

In [ ]:
# Enrichment of the IP fractions is calculated against the untagged samples in the same batch
adata_ip = adata[:, ~adata.var.covariate_Experiment.isin(["NOC", "PROTEOME"])].copy()
adata_ip_enr = gr.pp.calculate_enrichment_vs_untagged(
    adata_ip,
    covariates="covariate_Batch",
    subcellular_enrichment_column="covariate_Bait",
    untagged_name=".._UNTAGGED",
)

# adata_ip = adata_ip[:, adata_ip.var.subcellular_enrichment != "UNTAGGED"]
# adata_ip_enr = gr.pp.calculate_enrichment_vs_all(
#     adata_ip,
#     covariates="covariate_Batch",
#     subcellular_enrichment_column="covariate_Bait",
# )
adata_ip_enr


In [ ]:
# Enrichment of the NOC samples is calculated as a logFC
adata_noc = adata[:, adata.var.covariate_Experiment == "NOC"].copy()

# First average the Replicates
adata_noc_enr = gr.pp.aggregate_samples(
    adata_noc, grouping_columns=["subcellular_enrichment"], agg_func=np.mean
)

adata_noc_enr.X = adata_noc_enr.X - adata_noc_enr.X.mean(axis=1)[:, None]

adata_noc_enr


In [ ]:
# Finally recombine the enrichment data with the IP data
adata_enr = ad.concat(
    [adata_ip_enr, adata_noc_enr], axis=1, merge="first", uns_merge="first"
)
adata_enr


# Dimensionality Reduction

In [ ]:
sc.pp.scale(adata_enr)

In [ ]:
# We can use PCA to visualize
sc.pp.pca(adata_enr)
sc.pl.pca(adata_enr, color=["hein2024_component"], size=30)
# Note that the resolution does not look great. This is because on the OrgIP data 2PCs are not enough to resolve the variation in the dataset.

In [ ]:
sc.pp.neighbors(adata_enr, n_neighbors=10, use_rep="X")
sc.tl.umap(adata_enr, min_dist=0.1)

In [ ]:
# This simplified workflow reproduces most clusters of the original publication well. Slight differences are expected due to different preprocessing steps.
sc.pl.umap(adata_enr, color=["hein2024_component"], size=30)

## Annotation

In [ ]:
# Instead of comparing to the complete publication annotation, we can create an annotation by starting with a set of markers and propagating the labels to the rest of the data.
# We will use the ground truth created for the original publication.

sc.pl.umap(adata_enr, color=["hein2024_gt_component"], size=30)

In [ ]:
gr.tl.knn_annotation(
    adata_enr,
    gt_col="hein2024_gt_component",
    key_added="knn_annotation",
    min_probability=0.9,
)
sc.pl.umap(adata_enr, color=["knn_annotation"], size=30)

@Duo, we can remove the next section too, if confusing

In [ ]:
# Alternatively we can use unsupervised clustering to create an annotation.

sc.tl.leiden(adata_enr)
sc.pl.umap(adata_enr, color=["leiden"], size=30, legend_loc="on data")

In [ ]:
# The following section requires gseapy, which is not installed by default.
!pip install gseapy

In [ ]:
# These clusters can be annotated either manually or by using a set of markers.
enrichment_results = gr.tl.calculate_cluster_enrichment(
    adata_enr,
    cluster_key="leiden",
    gene_name_key="Gene_name_canonical",
    obs_key_added="leiden_top_enrichment",
    enrichment_ranking_metric="Combined Score",
)

sc.pl.umap(adata_enr, color=["leiden_top_enrichment"], size=30)